In [1]:
from joblib import dump, load
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [2]:
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [3]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [4]:
low_imp_features = load('/kaggle/input/mchmlmic-new/low_imp_features_new.joblib')

In [5]:
train = pd.read_parquet('/kaggle/input/mchmlmic-new/train.parquet.gzip')
train = train[~train.index.duplicated(keep='first')]
X = train.drop(pred_cols, axis=1)
y = train[pred_cols]

In [6]:
del(train)
import gc
gc.collect()

0

In [7]:
# drop the features with low importance and return a new dataframe
X = X.drop(columns=low_imp_features)
X

,Ec_median@CAS/IGSNRR/PML/V2&timestamped,Ei_median@CAS/IGSNRR/PML/V2&timestamped,Es_median@CAS/IGSNRR/PML/V2&timestamped,GPP_median@CAS/IGSNRR/PML/V2&timestamped,Ec_mean@CAS/IGSNRR/PML/V2&timestamped,Ei_mean@CAS/IGSNRR/PML/V2&timestamped,Es_mean@CAS/IGSNRR/PML/V2&timestamped,GPP_mean@CAS/IGSNRR/PML/V2&timestamped,qc_mean@CAS/IGSNRR/PML/V2&timestamped,Ec_max_max@CAS/IGSNRR/PML/V2&timestamped,...,onehotencoder__DHSREGNA_Somali,onehotencoder__DHSREGNA_Vlore,onehotencoder__DHSREGNA_atlantica,onehotencoder__DHSREGNA_lower egypt rural,onehotencoder__DHSREGNA_lower egypt urban,onehotencoder__DHSREGNA_somali,onehotencoder__DHSREGNA_upper egypt rural,remainder__DHSYEAR,remainder__LATNUM,remainder__LONGNUM
DHSID,,,,,,,,,,,,,,,,,,,,,
AL200800000001,0.697994,0.051240,0.720611,2.867798,1.004067,0.063028,0.703046,3.573272,0.000000,4.486412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.822650,19.838322
AL200800000002,0.546368,0.072052,0.641110,2.831818,0.949528,0.080869,0.636434,3.552526,0.000000,3.593682,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.696846,20.007555
AL200800000004,0.793127,0.056218,0.680800,3.167071,1.091989,0.066411,0.676971,3.901542,0.000000,3.753731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.798930,19.863338
AL200800000010,0.602147,0.047015,0.761851,2.234685,0.719552,0.056869,0.810261,2.521939,0.000000,3.266487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.698520,19.950300
AL200800000011,0.550234,0.037470,0.794295,1.932693,0.446101,0.026394,0.963341,1.428515,0.000000,2.894509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,40.717968,19.935875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW201500000395,1.291659,0.023959,0.600450,3.688477,1.502375,0.089699,0.486703,4.368353,0.907811,3.280793,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015.0,-17.166506,29.718370
ZW201500000396,0.988795,0.006372,0.622020,2.953615,0.819432,0.048184,0.867226,2.659858,0.567900,3.114433,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015.0,-17.915289,31.156115
ZW201500000397,1.225136,0.021697,0.694095,4.129716,1.276986,0.081176,0.686939,4.355009,0.328377,3.072238,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015.0,-18.379501,31.872288


In [8]:
test = pd.read_parquet('/kaggle/input/mchmlmic-new/test.parquet.gzip')
test = test.drop(columns=low_imp_features)

In [9]:
model = RandomForestRegressor(n_estimators=1000, max_features=0.5, random_state=42, n_jobs=-1, verbose=1)
model.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 64.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 147.9min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 264.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 334.2min finished


RandomForestRegressor(max_features=0.5, n_estimators=1000, n_jobs=-1,
                      random_state=42, verbose=1)

In [10]:
# dump(model, '/kaggle/working/test_rf.joblib')

In [11]:
y_pred = model.predict(test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    4.8s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    8.6s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    9.4s finished


In [12]:
out = pd.DataFrame(y_pred, columns=pred_cols)
out['DHSID'] = test.index
out = out[['DHSID'] + pred_cols]
out.to_csv('/kaggle/working/new_test.csv', index=False)

In [13]:
out

,DHSID,Mean_BMI,Median_BMI,Unmet_Need_Rate,Under5_Mortality_Rate,Skilled_Birth_Attendant_Rate,Stunted_Rate
0,AL200800000003,25.07296,24.39415,25.29114,4.13148,84.86637,13.59895
1,AL200800000005,25.24546,24.50825,34.78369,4.26397,76.99187,12.19720
2,AL200800000007,25.04639,24.71062,19.21375,4.55017,77.41384,13.92474
3,AL200800000008,25.61826,25.04812,16.75866,5.93739,72.22819,7.71270
4,AL200800000009,25.15251,24.99055,14.92203,4.99021,79.69471,8.92809
...,...,...,...,...,...,...,...
14995,ZW201500000382,23.74198,23.03044,20.49757,7.44992,71.89709,29.67159
14996,ZW201500000383,24.08255,23.42240,19.52946,7.03896,73.88015,30.92932
14997,ZW201500000386,25.66124,24.93243,24.53156,6.07599,83.51379,27.15404
14998,ZW201500000390,25.41365,24.61518,9.61603,4.92099,93.19639,23.73430
